# Carga de datos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

data = pd.read_csv("DelayedFlights.csv") # carga de datos
data.drop(["Unnamed: 0"], axis=1, inplace=True)
data.head(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [2]:
numero_inicial_filas = len(data)
data.shape

(1936758, 29)

# ¿Vuelos repetidos?

En el análisis exploratorio, se pudo ver que existen números de vuelo repetidos, pero ¿esto significa que existen vuelos repetidos?

Esta vez, en vez de analizar el número de vuelo por sí solo, definimos un conjunto de columnas que definen completamente a un vuelo, los cuales son:

- Número de vuelo FlightNum
- Year, Month y DayofMonth (que en conjunto dan la fecha del vuelo)
- CRSDepTime que es la hora programada de departure (recordar que la hora se encuentra en formato militar, que es un entero)
- Origin que corresponde al aeropuerto del origen
- Dest que corresponde al aeropuerto de destino
- UniqueCarrier que define la aerolínea que opera el vuelo

Esta combinación de columnas significa que que un vuelo se le asigna un número por fecha, hora de despegue, ruta (origen destino) y aerolínea

In [3]:
flight_def_cols = ["FlightNum", "Year", "Month", "DayofMonth", "CRSDepTime", "Origin", "Dest", "UniqueCarrier"]
print(f"Número de filas del dataset: {numero_inicial_filas}")
print(f"Número de vuelos diferentes: {len(data.drop_duplicates(subset=flight_def_cols))}")

Número de filas del dataset: 1936758
Número de vuelos diferentes: 1936756


Dado lo anterior, se quitan vuelos repetidos

In [4]:
data.drop_duplicates(subset=flight_def_cols, inplace=True)

Se puede ver que el número de vuelos diferentes es cercano al largo del dataset, pero no es igual por 2 filas, por lo cual se debe mantener este filtro para no considerar 2 veces el mismo vuelo

# Otros filtros relevantes
El objetivo de este dataset es analizar los factores que inciden en los retrasos, para lo cual se toman las siguientes consideraciones:

- Los vuelos cancelados no aportan con información, ya que no se realizaron y por ende solo aportan valores faltantes
- Los vuelos que tuvieron problemas de medición en el tiempo de llegada o ArrTime deben ser filtrados, ya que pueden tener un retraso importante y no se está capturando esa información
- Se vio en el análisis exploratorio que ciertos vuelos tenían un número negativo en la variable CRSElapsedTime, los cuales deben ser filtrados, ya que dicho valor no puede ser negativo y no se sabe si esto afectó en la medición de otras variables
- Se deben filtrar vuelos donde no existe la medición de retraso (tanto DepDelay y ArrDelay), ya que el objetivo es precisamente poder obtener información de los vuelos a la hora vs los vuelos retrasados. 

In [5]:
data = data[
    (data["Cancelled"] == 0)
    & ~(data["ArrTime"].isnull())
    & ~(data["DepTime"].isnull())
    & ~(data["CRSElapsedTime"].isnull())
    & (data["CRSElapsedTime"]>0)
    & ~(data["ArrDelay"].isnull())
    & ~(data["DepDelay"].isnull())
    ]

print(f"Longitud inicial del dataset: {numero_inicial_filas}")
print(f"Longitud del dataset después de los filtros: {len(data)}")
print("Proporción de datos post-filtros: {:.2f} %".format((len(data)/numero_inicial_filas*100)))


Longitud inicial del dataset: 1936758
Longitud del dataset después de los filtros: 1928368
Proporción de datos post-filtros: 99.57 %


Se puede ver que se conserva más del 99% de los datos después de aplicar estos filtros

# Columnas a eliminar
Según lo visto en el análisis exploratorio, los retrasos  'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay' no se pueden utilizar en los análisis, ya que poseen un número muy elevado de valores faltantes alcanzando hasta un 35%. Cabe destacar que otras variables como Airtime tienen una proporción de valores faltantes, pero no es necesario filtrarlos, ya que para la aplicación para la cual se propondrá este dataset, no se utilizan.